In [73]:
pronoun = "Me"

#vocab = ["I", "They", "We", "My", "Us", "You", "Me"]

# This file carries out TFIDF.
# A corpus is created based on pronouns and tweets.
# Each document in the corpus consists of tweets from a specified date which contain the specific pronoun.

In [74]:
# reference https://programminghistorian.org/en/lessons/analyzing-documents-with-tfidf#1-as-an-exploratory-tool-or-visualization-technique
# Accessed: 

In [75]:
from pathlib import Path

all_txt_files =[]
for file in Path("../March-No-Retweets/Pronoun/"+pronoun).rglob("*.txt"):
     all_txt_files.append(file.parent / file.name)
# counts the length of the list
n_files = len(all_txt_files)

In [76]:
all_txt_files.sort()
all_txt_files[0]

WindowsPath('../March-No-Retweets/Pronoun/Me/2020-03-25-pre-me-pronoun.txt')

In [77]:
all_docs = []
for txt_file in all_txt_files:
    with open(txt_file) as f:
        txt_file_as_string = f.read()
    all_docs.append(txt_file_as_string)

In [78]:
#import the TfidfVectorizer from Scikit-Learn.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=.65, min_df=2, stop_words=None, use_idf=True, norm=None)
transformed_documents = vectorizer.fit_transform(all_docs)

In [79]:
transformed_documents_as_array = transformed_documents.toarray()
# use this line of code to verify that the numpy array represents the same number of documents that we have in the file list
len(transformed_documents_as_array)

8

In [80]:
print(transformed_documents_as_array)

[[0.         0.         1.40546511 ... 1.58778666 0.         2.09861229]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.40546511 ... 3.17557333 0.         0.        ]
 ...
 [1.58778666 1.81093022 2.81093022 ... 0.         0.         0.        ]
 [1.58778666 0.         2.81093022 ... 0.         1.58778666 0.        ]
 [0.         1.81093022 0.         ... 1.58778666 1.58778666 0.        ]]


In [81]:
import pandas as pd

# make the output folder if it doesn't already exist
Path("../tf_idf_output").mkdir(parents=True, exist_ok=True)

# construct a list of output file paths using the previous list of text files the relative path for tf_idf_output
output_filenames = [str(txt_file).replace(".txt", ".csv").replace("txt/", "tf_idf_output/") for txt_file in all_txt_files]

all_list = []

# loop each item in transformed_documents_as_array, using enumerate to keep track of the current position
for counter, doc in enumerate(transformed_documents_as_array):
    # construct a dataframe
    tf_idf_tuples = list(zip(vectorizer.get_feature_names(), doc))
    one_doc_as_df = pd.DataFrame.from_records(tf_idf_tuples, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True).head(20)
    print(len(tf_idf_tuples))
    all_list.append(tf_idf_tuples)
    # output to a csv using the enumerated value for the filename
    # one_doc_as_df.to_csv(output_filenames[counter])

5622
5622
5622
5622
5622
5622
5622
5622


In [82]:
flat = []


for i in all_list:
    for j in i:
        flat.append(j)

all_doc_as_df = pd.DataFrame.from_records(flat, columns=['term', 'score']).sort_values(by='score', ascending=False).reset_index(drop=True).head(20)
all_doc_as_df.to_csv('../tf_idf_output/'+pronoun+'.csv')